### 模擬試験でのポイント

[**S3**]<br>

[**Amazon RDS DB**]<br>
- Amazon RDS DBインスタンスはリードレプリカを追加して、アプリケーションの読み込み負荷を分散することができる、これは非同期レプリケーションです。
- Optimesd Writeが利用できるMySQLバージョンに変更して、Optimesd Writeを適用して書き込み処理を高速化することも可能。Amazon RDSのOptimesd Writeは書き込み処理を最大50％向上できるRDSの機能
- マルチAZ構成でRDSマスターともう一つのAZへはRDAスレーブを配置して同期レプリケーションによる自動フェイルオーバー機能も追加可能
- Amazon RDSをAuroraサーバレス MySQLに変更することでピーク時にオートスケーリングはされますが、Auroraサーバレス はピーク以外の通常時においても一定のデータベース利用が発生しない、不規則なデータベース利用が発生する際に向いているDBタイプ
- クエリ処理には向いていますが、**ログ解析には向いていません**。

[**Amazon EBS**]<br>
- EBSのスループット最適化HDD
  - 高いスループットを必要とするアクセス頻度の高いワークロード向けの低コストの HDD ボリューム.プロビジョンドIOPS SSDよりも低コストで実現  
- EBSの汎用SSD
   - 幅広いトランザクションワークロードに対応できる価格とパフォーマンスのバランスが取れた汎用 SSD ボリューム
- EBSのプロビジョンドIOPS SSD
   - イテンシーの影響が大きいトランザクションワークロード向けに設計された極めてパフォーマンスの高い SSD ボリューム。コストを度外視して、最も高いスループット性能を求められる際には正解
- EBSのコールドHDD
   - アクセス頻度の低いワークロード向けに設計された極めて低コストの HDD ボリューム

[**DynamoDB**]<br>
- DynamoDBをオンデマンドに変更することで、一時的な負荷に対してキャパシティをスケーリングして高負荷に対応することが可能<br>
- (注意)*DynamoDBをマルチAZ構成で起動する機能はない*  
- オートスケーリングを利用したり、Amazon SQSを構成した方がコスト最適に対応。ただし「DynamoDBのオンデマンドモードを選択すると、DynamoDB は、前に到達したトラフィックレベルまで拡張または縮小して、ワークロードを即座に受け入れることができるようにします。したがって、**このモードにはAuto Scalingは必要ありません**」

[**CloudFront**]<br>
- CloudFrontは、コンテンツ配信ネットワーク（CDN）で、エンドユーザーに近いエッジロケーションからコンテンツを提供します。CloudFrontを使用することで、ウェブアプリケーションのレスポンス速度を向上させ、ユーザー体験を改善
- Amazon CloudFrontのフィールドレベル暗号化を実施して、アップロード時のデータの暗号化を実施することで、ユーザーによるデータアップロード時の暗号化に対するセキュリティレイヤーを追加することができます。さらに、フィールドレベル暗号化では、特定のデータに特定のアプリケーションのみがアクセスできるように制限することができる
- フィールドレベル暗号化により、ユーザーが機密情報をウェブサーバーに安全にアップロード
- (注意点)**AWS KMSを適用して、アップロード時の暗号化を実施することはできません。**
- CloudFrontディストリビューションを作成し、**オリジン（元となるサーバ）としてALBのDNS名を設定**します。これにより、CloudFrontはALBを通じてEC2インスタンスにリクエストをルーティングします。
- 必要に応じて、キャッシュ設定や、SSL証明書（HTTPS通信を使用する場合）の設定を行います。
- **エッジロケーションによるファイル圧縮処理を実施する。ことで経費削減が可能**

[**AWS Global Accelerator**]<br>
- パブリックアプリケーションの可用性、パフォーマンス、およびセキュリティの向上させるためのネットワークサービス
- エッジでパケットをプロキシして最適なエンドポイントグループにルーティングするかを決めることで、TCP または UDP を利用した幅広いアプリケーションのパフォーマンスを向上
  
[**SQS**]<br>
- SQSとAuto Scalingを構成する際はCloudWatchメトリックスに基づいてキューの処理量に応じたスケーリングを設定
- 完全マネージド型のメッセージキューイングサービスで、マイクロサービス、分散システム、およびサーバーレスアプリケーションの切り離しとスケーリングが可能
- 負荷分散が可能

[**Amazon SNS**]<br>
- イベント通知を起点としたトリガーとして利用し、実処理はSQSに連携してポーリングするというのが標準的なAWSでのアーキテクチャ構成
- コンポーネント処理間の連携を確実に実行するには並列処理による負荷分散が可能なAmazon SQSが最適


[**Amazon Kinesis Data Streams**]<br>
- デバイスなどから送られる大量のストリーミングデータをリアルタイムに収集、処理して別のサービスに送信するサービス
- **ログ解析には向いていません。**

[**Amazon Kinesis Data Kinesis Firehose**]<br>
- 大量のストリーミングデータを取得して、Lambda関数によってETL処理をしつつ、RedShiftやS3などのストレージやデータベースに配信するサービス

[**Amazon SWF**]<br>
クラウドのワークフロー管理アプリケーションで、複数マシン間でアプリケーションを連携させるためのツールを開発者に提供

[**Amazon EMR**]<br>
- Apache Sparkなどのソフトウェアを構成して、デバイスなどから送られる大量のストリーミングデータをリアルタイムに収集、分析するサービス
- Amazon EMRは、動的にスケーラブルなAmazon EC2インスタンス全体で膨大な量のデータ処理を簡単、高速、そして費用対効果の高いものにするマネージド**Hadoopフレームワーク**を提供します.
- Webインデックス作成、データマイニング、ログファイル分析、機械学習にむいてる

[**Amazon Inspector**]<br>
- 自動化されたセキュリティ評価サービスで、AWS にデプロイしたアプリケーションのセキュリティとコンプライアンスを向上させることができます。EC2インスタンスにエージェントをインストールして、そこにホストされたアプリケーションの意図しないネットワークエクスポージャーなどの脆弱性を検知

[**Amazon GuardDuty**]<br>
- Amazon S3 ログ、CloudTrail 管理イベントログ、DNS ログ、Amazon EBS ボリュームデータ、Kubernetes Audit Logs、Amazon VPC フローログ、RDS ログインアクティビティの AWS CloudTrail データイベントなどのデータソースを分析して、DDoS攻撃以外の様々なネットワークへの不正アクセスを自動的に検知

[**AWS Compute Optimize**]<br>
- EC2インスタンスタイプ、Amazon Elastic Block Store (EBS) ボリューム、AWS Lambda 関数の 3 種類の AWS リソースの過剰プロビジョニングや過小プロビジョニングなどの利用状況に基づいて、コスト最適化のアドバイスを実施

[**Amazon ALB**]<br>
- ALBは**レイヤー7の負荷分散を行い、アプリケーションレベルでのルーティングや、ヘルスチェックをサポート**します。
- 



### 知見
[**CloudFrontとHTTP領域**]<br>
CloudFrontを利用しグローバルにコンテンツを配信することを考えた場合「言語処理」
を例えば以下のようにHTTP領域にて展開する方法もある。パラメータをこれで送ることが可能
- http://pintor.cloudfront.net/index?language=jp
- http://pintor.cloudfront.net/index?language=en<br>
この場合は「クエリ文字列パラメーター値に基づくキャッシュ設定を行うことでHTTPフィールド内の言語表示に基づいて配信方法を切り替えること」で実現可能

(補足)<br>
クエリ文字列（URLパラメーター）とは、サーバーに情報を送るためにURLの末尾につけ足す文字列（変数）のことです。「?」をURLの末尾につけ、その次に「パラメーター＝値」をつけ、複数のパラメーターをつけたい場合は「&」を使用します。この形式で、サーバーに送信したいデータをURLにつけ加えることが可能

[**SQSによるマイクロサービス化されたコンポーネントとの連携処理**]<br>
マイクロサービス化されたコンポーネント間の処理を連携する際は、Amazon SQSを利用したポーリング処理が最適となります。
例えば：Amazon ECSを活用してマイクロサービス型のアプリケーションを構築しています。 このアプリケーションのアーキテクチャでは、マイクロサービス１とマイクロサービス２に分割し、マイクロサービス１の処理後にマイクロサービス２に処理が引き渡されます。
]

[**IoTセンサーに基づいたデータ分析**]<br>
上記のようなケースではまさに「Amazon Kinesis Data Streams」を活用すべき。Amazon Kinesis Data Firehose経由でAmazon S3バケットに送信。<br>
注意点として「デフォルトでは、データレコードがAmazon Kinesis Data Streamsに投入されてから24時間以内までアクセスできるように設定されている。」

[**ELBから結合されたログファイルを分析できるようにアプリケーションを構築**]<br>
Amazon S3に対してELBのログファイルを収集し、Amazon EMRでログ解析を実施する方法が考えられる。
ログファイルの収集場所としてはAmazon S3が適切。このログファイル分析には大量データ解析が可能なパフォーマンスが必要

[**エッジ処理とは？**]
「エッジ処理」とは、CloudFrontエッジロケーションで実行される処理のことを指します。エッジロケーションでは、リクエストの処理が行われる際、以下のような処理が実施されることがあります：
- コンテンツキャッシュ: CloudFrontエッジサーバーが、ウェブサイトの静的コンテンツ（HTML、画像、動画、CSS、JavaScriptなど）をキャッシュし、リクエストがあるたびに高速に返します。これにより、リクエスト元のサーバー（オリジンサーバー）への負荷を軽減し、応答速度を向上させます。
- リクエスト/レスポンスのカスタマイズ: CloudFrontは、Lambda@Edgeという機能を使って、リクエストやレスポンスに対してカスタマイズを行うこともできます。これにより、リダイレクトや認証、リクエストヘッダーの変更、レスポンス内容の変更など、エッジでの処理を行えます。
- 動的コンテンツの処理: 一部の動的コンテンツ（例えば、ユーザーごとに異なるデータを提供するAPIレスポンスなど）も、エッジサーバーで処理され、オリジンサーバーへのアクセスが減少します。これにより、コンテンツ配信が迅速になります。


### コーディングによる解説